In [59]:
import pandas as pd
import json
from pathlib import Path

df = pd.read_csv('/home/capre/Point-Cloud-Stream/invalid.csv')
# prompt,folder,mark

path = Path("/home/capre/disk_4/yutao/data")
res_path = path / "resources"
count = 0
cout_2 = 0
json_contents = {}
for json_f in path.rglob("sa*.json"):
    file_contents = json.loads(json_f.read_text())
    for each_f in file_contents:
        file_contents[each_f]['folder'] = json_f.parent.name
        # file_contents[each_f]['prompt']

        if df[(df['prompt'] == file_contents[each_f]['prompt']) & (df['folder'] == json_f.parent.name)].shape[0] == 0:
            count += 1
            continue
        cout_2 +=1
        json_contents[each_f] = file_contents[each_f]
        json_contents[each_f]['mark'] = 'invalid'
        # json_contents[each_f]['folder'] = json_f.parent.name
print(count)
dddf = pd.DataFrame(json_contents).T
dddf = dddf.rename({'folder': 'folder_name'}, axis=1)
dddf['uuid'] = dddf.index
dddf[['folder_name', 'uuid', 'mark']].to_csv('invalid_new.csv')
# dddf

FileNotFoundError: [Errno 2] No such file or directory: '/home/capre/Point-Cloud-Stream/invalid.csv'

# remove invalid files

In [60]:
import pandas as pd
from pathlib import Path

path_csv_dir = Path('/home/capre/Point-Cloud-Stream/')
df = pd.DataFrame()
for each in path_csv_dir.glob("*.csv"):
    df = pd.concat([df, pd.read_csv(each)], ignore_index=True)

invalid_list = df[df['mark']=='Invalid']['uuid'].to_list()

path = Path("/home/capre/disk_4/yutao/data")
res_path = path / "resources"

count = 0
for file in res_path.glob("*"):
    for each_invalid in invalid_list:
        if each_invalid in file.name:
            file.unlink()
            print(f"Deleted: {file}")
            count += 1
            break

df

,folder_name,uuid,mark
0,lincheng 3.3,0a66645b5f4746f89d27ffa8249820ea,Invalid
1,lincheng 3.3,15d0d84f5b774c24a6b0e202c4eb05c8,Invalid
2,lincheng 3.3,c84ec2023a2f4d28a822fe8910c60227,Invalid
3,lincheng 3.3,69805b093b774a4d8da820f9f5e0b615,Invalid
4,lincheng 3.3,073cda272aa24c1697248fc62664fea5,Invalid
...,...,...,...
587,wangyi,fc7d49aca49a42149db2192a84b1113d,Invalid
588,wangyi,2517a5b72bf943bdabd429c9c69487a7,Invalid
589,wangyi,5a7baf7aadb94c6e8c4438bda05192a7,Invalid
590,wangyi,d572ad4c8b544d23b3acd91f68457366,Invalid


# check prompt

In [67]:
import pandas as pd
from pathlib import Path
import json

path_csv_dir = Path('/home/capre/Point-Cloud-Stream/')
df = pd.DataFrame()
for each in path_csv_dir.glob("*.csv"):
    df = pd.concat([df, pd.read_csv(each)], ignore_index=True)

invalid_list = df[df['mark']=='Invalid']['uuid'].to_list()

path = Path("/home/capre/disk_4/yutao/data")
res_path = path / "resources"

json_contents = {}
for json_f in path.rglob("sa*.json"):
    file_contents = json.loads(json_f.read_text())
    for each_f in file_contents:
        # print(each_f)
        valid = 1
        if each_f in invalid_list:
            valid = 0
            continue
        if len(file_contents[each_f]['pose_record']) == 0:
            valid = 0
            continue
        for each_color in file_contents[each_f]['color_files']:
            if not Path(res_path / each_color).exists():
                valid = 0
                continue
        
        if valid == 1:
            json_contents[each_f] = file_contents[each_f]
            json_contents[each_f]['folder'] = json_f.parent.name

print(len(json_contents))
json.dump(json_contents, open('valid.json', 'w'))


281


In [ ]:
df

In [68]:
dff = pd.DataFrame(json_contents).T
dff[['prompt', 'folder']].to_csv("valid.csv")
count_folder = dff.value_counts('folder')
count_folder
dff

,prompt,pose,pose_record,t_base_to_cam,joint_positions,joint_position_records,poses,color_files,depth_files,point_cloud_files,folder
a57cc390c718448e9cbe9df44812f273,Please reach to the inner and bottom position ...,"[[0.011799790793634142, -0.29289599575818753, ...","[[0.1176599961949988, -0.31034248620678956, 0....","[[0.7764385466262372, -0.6299549384999015, -0....","[[-1.2364843888350696, -1.3190317299831658, -1...","[[-0.9596967304209544, -1.4996579677567, -1.75...","[{'pose0': [0.011799790793634142, -0.292895995...","[a57cc390c718448e9cbe9df44812f273_color_0.png,...","[a57cc390c718448e9cbe9df44812f273_depth_0.npy,...",[],chengchangran
8ce891da93614ea88b2783c3ac98baa9,Please reach to the outer and bottom position ...,"[[0.1820476754667187, -0.27202313190543265, 0....","[[0.18174501274694532, -0.2721600118008799, 0....","[[0.7764385466262372, -0.6299549384999015, -0....","[[1.5800002115222105, -1.7040609077424616, 1.5...","[[1.578849725931296, -1.7040609077424616, 1.53...","[{'pose0': [0.1820476754667187, -0.27202313190...","[8ce891da93614ea88b2783c3ac98baa9_color_0.png,...","[8ce891da93614ea88b2783c3ac98baa9_depth_0.npy,...",[],pyy3.3
1b69e872183546beb178ee3e9ae1e9cb,Please reach to the outer and bottom position ...,"[[0.1848050594060465, -0.2859098146237814, 0.4...","[[0.1848296378066547, -0.28586929744417217, 0....","[[0.7764385466262372, -0.6299549384999015, -0....","[[1.6942817802196908, -1.702431053155333, 1.60...","[[1.6943776540189337, -1.702431053155333, 1.60...","[{'pose0': [0.1848050594060465, -0.28590981462...","[1b69e872183546beb178ee3e9ae1e9cb_color_0.png,...","[1b69e872183546beb178ee3e9ae1e9cb_depth_0.npy,...",[],pyy3.3
b443eeb1cd0d43d3a6ef2ae74cd856da,Please reach to the outer and bottom position ...,"[[0.058949435000263145, -0.34253884247559635, ...","[[0.05907853645381568, -0.3425783298296326, 0....","[[0.7764385466262372, -0.6299549384999015, -0....","[[1.227184630308513, -1.6675329902309348, 1.59...","[[1.2275681255054844, -1.6676288640301775, 1.5...","[{'pose0': [0.058949435000263145, -0.342538842...","[b443eeb1cd0d43d3a6ef2ae74cd856da_color_0.png,...","[b443eeb1cd0d43d3a6ef2ae74cd856da_depth_0.npy,...",[],pyy3.3
7def2ec798e24823a2147b341cafda2d,Please reach to the outer and bottom position ...,"[[0.2502331300540962, -0.19646077819032232, 0....","[[0.2502499485344978, -0.196374319189288, 0.45...","[[0.7764385466262372, -0.6299549384999015, -0....","[[1.919009965644937, -1.7798012091443152, 1.54...","[[1.9192975870426656, -1.7798012091443152, 1.5...","[{'pose0': [0.2502331300540962, -0.19646077819...","[7def2ec798e24823a2147b341cafda2d_color_0.png,...","[7def2ec798e24823a2147b341cafda2d_depth_0.npy,...",[],pyy3.3
...,...,...,...,...,...,...,...,...,...,...,...
a3d4b4f2ccc349b6955ae8ad0c7ae683,Please reach to the outer and upper position o...,"[[0.1993420157635141, -0.3177348380052986, 0.3...","[[0.19870207946026813, -0.31812307069899354, 0...","[[0.7764385466262372, -0.6299549384999015, -0....","[[-0.6618168361734114, -1.5949565242040955, -1...","[[-0.6637343121582684, -1.5948606504048526, -1...","[{'pose0': [0.1993420157635141, -0.31773483800...","[a3d4b4f2ccc349b6955ae8ad0c7ae683_color_0.png,...","[a3d4b4f2ccc349b6955ae8ad0c7ae683_depth_0.npy,...",[],liukaishuo
93e88ba2320e4a36a79cfff47867e9d7,Please reach to the outer and upper position o...,"[[0.23996309694906823, -0.29089687180354823, 0...","[[0.24014933100288743, -0.2906977216786685, 0....","[[0.7764385466262372, -0.6299549384999015, -0....","[[-0.5338253141842031, -1.6254443923633226, -1...","[[-0.5329624499910175, -1.6253485185640797, -1...","[{'pose0': [0.23996309694906823, -0.2908968718...","[93e88ba2320e4a36a79cfff47867e9d7_color_0.png,...","[93e88ba2320e4a36a79cfff47867e9d7_depth_0.npy,...",[],liukaishuo
7aaed20481d64a52819414abe3bf5c12,Please reach to the outer and upper position o...,"[[0.1211319168465088, -0.3557142574482251, 0.3...","[[0.12102972419516742, -0.3556852819261871, 0....","[[0.7764385466262372, -0.6299549384999015, -0....","[

In [69]:
dff['real_prompt'] = dff['prompt'].apply(lambda x: x.split('.')[0])
dff['real_prompt'].value_counts()
dff.shape[0]

281

In [ ]:
# Please reach to the outer and bottom position of the nearest breast. 98
# Please reach to the inner and bottom position of the nearest breast. 24
# Please reach to the inner and  upper position of the nearest breast. 48
# Please reach to the outer and upper position of the nearest breast. 39
# Please reach to the outer position of the nearest breast. 0
# Please reach to the inner position of the nearest breast. 21
# Please reach to the upper position of the nearest breast. 0
# Please reach to the bottom position of the nearest breast. 51

# remove no record files

In [58]:
import json
from pathlib import Path

path = Path("/home/capre/disk_4/yutao/data")

file_name = []
for json_f in path.rglob("sa*.json"):
    json_contents = json.loads(json_f.read_text())
    print(json_f.parent.name, len(json_contents))
    for each in json_contents:
        file_name.extend(json_contents[each]['color_files'])
        file_name.extend(json_contents[each]['depth_files'])
    # file_contents.update(json_contents)
    # break
print(len(file_name))

res_path = path / "resources"

not_exist_count = 0
exist_count = 0
for file in res_path.glob("*"):
    if file.name not in file_name:
        not_exist_count += 1
        # Delete the file if it does not exist in the file_name list
        file.unlink()
        print(f"Deleted: {file}")
    else:
        exist_count += 1
print(exist_count, not_exist_count)


chengchangran 4
chengchen 12
pyy3.3 40
pyy3.5 27
wangxiqin 40
yijun 39
zengli2 1
zhangxian 40
panyingying 80
wangxiqin3.1 39
lincheng 3.3 40
yy2025.3.4 37
guoyutong3.2 40
wangyijun3.1 40
lincheng2.19 78
zhangxian3.4 40
zengli 22
guoyutong 13
wangyi 50
liukaishuo 17
161466
78312 0


# yolo finetun using different dataset

In [ ]:
from tqdm import tqdm

from ultralytics.utils.checks import check_requirements
from ultralytics.utils.downloads import download
from ultralytics.utils.ops import xyxy2xywhn

import numpy as np
from pathlib import Path

check_requirements(('pycocotools>=2.0',))
from pycocotools.coco import COCO

# Make Directories
dir = Path('/home/capre/disk_4/yutao/ultralytics/datasets/Objects365')  # dataset root dir
for p in 'images', 'labels':
    (dir / p).mkdir(parents=True, exist_ok=True)
    for q in 'train', 'val':
        (dir / p / q).mkdir(parents=True, exist_ok=True)

# Train, Val Splits
for split, patches in [('train', 50 + 1), ('val', 43 + 1)]:
    print(f"Processing {split} in {patches} patches ...")
    images, labels = dir / 'images' / split, dir / 'labels' / split

    # Download
    url = f"https://dorc.ks3-cn-beijing.ksyun.com/data-set/2020Objects365%E6%95%B0%E6%8D%AE%E9%9B%86/{split}/"
    if split == 'train':
        download([f'{url}zhiyuan_objv2_{split}.tar.gz'], dir=dir)  # annotations json
        download([f'{url}patch{i}.tar.gz' for i in range(patches)], dir=images, curl=True, threads=8)
    elif split == 'val':
        download([f'{url}zhiyuan_objv2_{split}.json'], dir=dir)  # annotations json
        download([f'{url}images/v1/patch{i}.tar.gz' for i in range(15 + 1)], dir=images, curl=True, threads=8)
        download([f'{url}images/v2/patch{i}.tar.gz' for i in range(16, patches)], dir=images, curl=True, threads=8)

    # Move
    for f in tqdm(images.rglob('*.jpg'), desc=f'Moving {split} images'):
        f.rename(images / f.name)  # move to /images/{split}

    # Labels
    coco = COCO(dir / f'zhiyuan_objv2_{split}.json')
    names = [x["name"] for x in coco.loadCats(coco.getCatIds())]
    for cid, cat in enumerate(names):
        catIds = coco.getCatIds(catNms=[cat])
        imgIds = coco.getImgIds(catIds=catIds)
        for im in tqdm(coco.loadImgs(imgIds), desc=f'Class {cid + 1}/{len(names)} {cat}'):
            width, height = im["width"], im["height"]
            path = Path(im["file_name"])  # image filename
            try:
                with open(labels / path.with_suffix('.txt').name, 'a') as file:
                    annIds = coco.getAnnIds(imgIds=im["id"], catIds=catIds, iscrowd=None)
                    for a in coco.loadAnns(annIds):
                        x, y, w, h = a['bbox']  # bounding box in xywh (xy top-left corner)
                        xyxy = np.array([x, y, x + w, y + h])[None]  # pixels(1,4)
                        x, y, w, h = xyxy2xywhn(xyxy, w=width, h=height, clip=True)[0]  # normalized and clipped
                        file.write(f"{cid} {x:.5f} {y:.5f} {w:.5f} {h:.5f}\n")
            except Exception as e:
                print(e)


In [ ]:
import os
from ultralytics import YOLO
# os.chdir("/home/capre/disk_4/yutao/ultralytics")
# Load a model
model = YOLO("/home/capre/Point-Cloud-Stream/runs/segment/train6/weights/best.pt")  # load a pretrained model (recommended for training)

# Train the model
# results = model.train(data="Objects365.yaml", epochs=100, imgsz=640)
image_path = '/home/capre/disk_4/yutao/data/resources/0a7ee4d0cf344e15a81c68be0be1fb96_color_2.png'
res= model.predict(source=image_path, save=False, verbose=False)
res[0].show()

# yolo finetune for breast seg

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11x-seg.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="/home/capre/disk_4/yutao/breast-seg/dataset3/dataset.yaml", epochs=100, imgsz=640)

# masks filename align

In [ ]:
import os

def add_leading_zeros_to_masks(masks_dir, total_digits=4):
    """
    将 masks_dir 中的文件名加上前导零，使其成为指定长度的数字字符串。

    参数：
    - masks_dir: 掩码文件夹的路径
    - total_digits: 文件名应达到的总位数，默认为4
    """
    for filename in os.listdir(masks_dir):
        # 分离文件名和扩展名
        basename, extension = os.path.splitext(filename)
        try:
            # 将文件名转换为整数，以确保文件名是数字
            number = int(basename)
            # 格式化新的文件名，添加前导零
            new_basename = f"{number:0{total_digits}d}"
            new_filename = new_basename + extension
            # 构建完整的源和目标路径
            src = os.path.join(masks_dir, filename)
            dst = os.path.join(masks_dir, new_filename)
            # 重命名文件
            os.rename(src, dst)
            print(f"重命名：{filename} -> {new_filename}")
        except ValueError:
            print(f"跳过非数字文件名：{filename}")

# 使用示例：
masks_dir = '/home/capre/disk_4/yutao/breast-seg/masks'  # 替换为您的 masks 文件夹路径
add_leading_zeros_to_masks(masks_dir)


# yolo dataset make

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import shutil

# Define paths
images_dir = '/home/capre/disk_4/yutao/breast-seg/images'  # Replace with your images directory
masks_dir = '/home/capre/disk_4/yutao/breast-seg/masks'      # Replace with your masks directory
dataset_root = '/home/capre/disk_4/yutao/breast-seg/dataset2'  # Replace with your dataset root directory

# Create necessary directories
os.makedirs(os.path.join(dataset_root, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'labels', 'val'), exist_ok=True)

# Get list of image files
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Shuffle and split into train and val
random.shuffle(image_files)
split_index = int(len(image_files) * 0.8)
train_files = image_files[:split_index]
val_files = image_files[split_index:]

def process_dataset(phase, files):
    for image_file in files:
        # Read image and mask
        image_path = os.path.join(images_dir, image_file)
        mask_path = os.path.join(masks_dir, image_file)  # Assuming mask has the same name
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Check if mask exists
        if mask is None:
            print(f"Mask not found for image {image_file}")
            continue

        height, width = mask.shape
        # Threshold mask to binary
        _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

        # Find contours
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Create label file
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(dataset_root, 'labels', phase, label_file)

        with open(label_path, 'w') as f:
            for contour in contours:
                # Simplify contour
                epsilon = 0.001 * cv2.arcLength(contour, True)
                contour = cv2.approxPolyDP(contour, epsilon, True)

                # Flatten contour array
                contour = contour.squeeze()
                if contour.ndim != 2:
                    continue  # Skip if contour is not 2D

                # Normalize coordinates
                normalized_contour = contour.astype(np.float32)
                normalized_contour[:, 0] /= width
                normalized_contour[:, 1] /= height

                # Flatten and convert to list
                contour_list = normalized_contour.flatten().tolist()

                # Write to file (class index is 0)
                line = '0 ' + ' '.join(map(str, contour_list))
                f.write(line + '\n')

        # Copy image to dataset folder
        shutil.copy(image_path, os.path.join(dataset_root, 'images', phase, image_file))

        # Visualization (optional)
        # if random.random() < 0.05:  # Adjust the probability as needed
        #     # Plot image and contours
        #     plt.figure(figsize=(10, 10))
        #     plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #     for contour in contours:
        #         contour = contour.squeeze()
        #         plt.plot(contour[:, 0], contour[:, 1], linewidth=2)
        #     plt.title(f"{phase} - {image_file}")
        #     plt.axis('off')
        #     plt.show()

# Process train and val datasets
process_dataset('train', train_files)
process_dataset('val', val_files)

# Generate dataset YAML file
dataset_yaml = os.path.join(dataset_root, 'dataset.yaml')
with open(dataset_yaml, 'w') as f:
    f.write(f"path: {dataset_root}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write("test: \n\n")
    f.write("names:\n")
    f.write("  0: object\n")


# add label from predict

In [ ]:
import os
import cv2
import numpy as np
import random
import shutil
from ultralytics import YOLO

# Define paths
images_dir = '/home/capre/disk_4/yutao/breast-seg/images'  # Replace with your images directory
masks_dir = '/home/capre/disk_4/yutao/breast-seg/masks'      # Replace with your masks directory
dataset_root = '/home/capre/disk_4/yutao/breast-seg/dataset3'  # Replace with your dataset root directory

# Create necessary directories
os.makedirs(os.path.join(dataset_root, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'labels', 'val'), exist_ok=True)

# Get list of image files
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Shuffle and split into train and val
random.shuffle(image_files)
split_index = int(len(image_files) * 0.8)
train_files = image_files[:split_index]
val_files = image_files[split_index:]

# Load the YOLO model
model = YOLO('yolo11x-seg.pt')  # Replace with the correct path to your model

# Get class names and assign a new class ID for your mask label
class_names = model.names  # A dictionary {class_id: class_name}
max_class_id = max(class_names.keys())
mask_class_id = max_class_id + 1
class_names[mask_class_id] = 'breast'  # Replace 'breast' with your class name

def process_dataset(phase, files):
    for image_file in files:
        # Read image and mask
        image_path = os.path.join(images_dir, image_file)
        mask_path = os.path.join(masks_dir, image_file)  # Assuming mask has the same name
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Check if mask exists
        if mask is None:
            print(f"Mask not found for image {image_file}")
            continue

        original_height, original_width = image.shape[:2]

        # Run the model to get predictions
        results = model.predict(source=image, save=False, verbose=False)
        result = results[0]

        # Create label file
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(dataset_root, 'labels', phase, label_file)

        with open(label_path, 'w') as f:
            # Process predicted masks
            if hasattr(result, 'masks') and result.masks is not None:
                pred_masks = result.masks.data.cpu().numpy()
                pred_classes = result.boxes.cls.cpu().numpy().astype(int)
                for mask_pred, class_id in zip(pred_masks, pred_classes):
                    # Convert mask to binary image
                    mask_pred = (mask_pred > 0.5).astype(np.uint8) * 255

                    # Resize mask back to original image size
                    mask_pred_resized = cv2.resize(mask_pred, (original_width, original_height), interpolation=cv2.INTER_NEAREST)

                    # Find contours
                    contours, _ = cv2.findContours(mask_pred_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                    for contour in contours:
                        # Simplify contour
                        epsilon = 0.001 * cv2.arcLength(contour, True)
                        contour = cv2.approxPolyDP(contour, epsilon, True)

                        # Flatten contour array
                        contour = contour.squeeze()
                        if contour.ndim != 2:
                            continue  # Skip if contour is not 2D

                        # Normalize coordinates
                        normalized_contour = contour.astype(np.float32)
                        normalized_contour[:, 0] /= original_width
                        normalized_contour[:, 1] /= original_height

                        # Flatten and convert to list
                        contour_list = normalized_contour.flatten().tolist()

                        # Write to file
                        line = f"{class_id} " + ' '.join(map(str, contour_list))
                        f.write(line + '\n')
            else:
                print(f"No predicted masks for image {image_file}")

            # Process your own mask
            # Threshold mask to binary
            _, mask_bin = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

            # Find contours
            contours, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            for contour in contours:
                # Simplify contour
                epsilon = 0.001 * cv2.arcLength(contour, True)
                contour = cv2.approxPolyDP(contour, epsilon, True)

                # Flatten contour array
                contour = contour.squeeze()
                if contour.ndim != 2:
                    continue  # Skip if contour is not 2D

                # Normalize coordinates
                normalized_contour = contour.astype(np.float32)
                normalized_contour[:, 0] /= original_width
                normalized_contour[:, 1] /= original_height

                # Flatten and convert to list
                contour_list = normalized_contour.flatten().tolist()

                # Write to file with mask_class_id
                line = f"{mask_class_id} " + ' '.join(map(str, contour_list))
                f.write(line + '\n')

        # Copy image to dataset folder
        shutil.copy(image_path, os.path.join(dataset_root, 'images', phase, image_file))

# Process train and val datasets
process_dataset('train', train_files)
process_dataset('val', val_files)

# Generate dataset YAML file
dataset_yaml = os.path.join(dataset_root, 'dataset.yaml')
with open(dataset_yaml, 'w') as f:
    f.write(f"path: {dataset_root}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write("test: \n\n")
    f.write("names:\n")
    for class_id in sorted(class_names.keys()):
        f.write(f"  {class_id}: {class_names[class_id]}\n")


# LEO Dataset Processing Pipeline

# Original Data Process (for xyz rxyz is invert)

In [ ]:
from pathlib import Path
import json
import numpy as np
from scipy.spatial.transform import Rotation as R
import pandas as pd
data_path = '/home/capre/disk_4/yutao/leo_data/data_2nd'


# def switch_xyzrxyz(rxyzxyz:list):
#     r_xyz, xyz = rxyzxyz[0:3], rxyzxyz[3:6]
#     xyzrxyz = np.hstack((xyz, r_xyz))
#     return xyzrxyz.tolist()

# print(switch_xyzrxyz([1,2,3,4,5,6]))

# def transform_pose(pose: list, T_cam_to_base):
#     pose: np.ndarray = np.array(pose)
#     t_xyz = pose[0:3]
#     r_xyz = pose[3:6]
#     rotation_matrix = R.from_euler('xyz', r_xyz.reshape(1, 3), degrees=False).as_matrix().reshape(3, 3)
#     T_end_to_base = np.eye(4)
#     T_end_to_base[:3, :3] = rotation_matrix
#     T_end_to_base[:3, 3] = t_xyz.ravel()
#     T_base_to_cam = np.linalg.inv(T_cam_to_base)

#     T_cam_to_end = T_base_to_cam @ T_end_to_base
#     new_t = T_cam_to_end[:3, 3]
#     new_r = R.from_matrix(T_cam_to_end[:3, :3]).as_euler('xyz', degrees=False)
#     xyzrxrzry = np.hstack((new_r, new_t.reshape(-1)))
#     return xyzrxrzry.tolist()


path = Path(data_path)
resources_path: Path = path / 'resources'
all_data = {}
x_values, y_values, z_values = [], [], []


import re
pattern = re.compile(r'(\d+$)')


for json_f in path.rglob("*saved_data.json"):
    # related_calib_json = json_f.parent / 'Calibration_results' / 'calibration_results.json'
    # if related_calib_json.exists():
        # calib = json.load(open(related_calib_json))
    saved_data = json.load(open(json_f))
    save = True
    for each_record in saved_data:
        prompt = saved_data[each_record]['prompt']
        if pattern.search(prompt):
            print(pattern.sub("", prompt))
            saved_data[each_record]['prompt'] = pattern.sub("", prompt)
            
        for each_image in saved_data[each_record]['color_files']:
            if not resources_path.joinpath(each_image).exists():
                save = False
                break
        if not save:
            break
        
        # for idx, pose in enumerate(saved_data[each_record]['poses']):
        #     saved_data[each_record]['poses'][idx]['pose0'] = switch_xyzrxyz(saved_data[each_record]['poses'][idx]['pose0'])
        #     if saved_data[each_record]['poses'][idx]['pose1'] is not None:
        #         saved_data[each_record]['poses'][idx]['pose1'] = switch_xyzrxyz(saved_data[each_record]['poses'][idx]['pose1'])


        for idx, pose in enumerate(saved_data[each_record]['pose']):
            # saved_data[each_record]['pose'][idx] = switch_xyzrxyz(pose)
            x_values.append(saved_data[each_record]['pose'][idx][0])
            y_values.append(saved_data[each_record]['pose'][idx][1])
            z_values.append(saved_data[each_record]['pose'][idx][2])
        # print(saved_data[each_record])
        all_data[each_record] = saved_data[each_record]
        # break

# Compute min and max for x, y, z
min_x, max_x = min(x_values), max(x_values)
min_y, max_y = min(y_values), max(y_values)
min_z, max_z = min(z_values), max(z_values)

# Print the results
print(f"Min x: {min_x}, Max x: {max_x}")
print(f"Min y: {min_y}, Max y: {max_y}")
print(f"Min z: {min_z}, Max z: {max_z}")


# map id to real prompts

In [ ]:
prompts_map = pd.read_csv(path / 'task_list.csv')
prompts_map['ID'] = prompts_map['ID'].astype(int).astype(str)

for each_record in all_data:
    all_data[each_record]['prompt'] = prompts_map[prompts_map['ID'] == all_data[each_record]['prompt']]['Prompts'].values[0]

# select data

In [ ]:
import random

all_data_len = len(all_data)
random_keys = random.sample(list(all_data.keys()), all_data_len)
all_data = {key: all_data[key] for key in random_keys}
print(len(all_data))
json.dump(all_data, open(path / 'all_data.json', 'w'), indent=4)

half_data_len = len(all_data) // 2
random_keys = random.sample(list(all_data.keys()), half_data_len)
half_data = {key: all_data[key] for key in random_keys}
print(len(half_data))
json.dump(half_data, open(path / 'half_data.json', 'w'), indent=4)

half_half_data_len = len(all_data) // 2 // 2
random_keys = random.sample(list(all_data.keys()), half_half_data_len)
half_half_data = {key: all_data[key] for key in random_keys}
print(len(half_half_data))
json.dump(half_half_data, open(path / 'half_half_data.json', 'w'), indent=4)

# batch process seg pcd from yolo

In [ ]:
import os
from pathlib import Path
from ultralytics import YOLO
import numpy as np
from plyfile import PlyElement, PlyData
import open3d as o3d
import open3d.core as o3c
import json
import torch
import cv2
import torch.utils.dlpack
from typing import List
from tqdm import tqdm
from app.utils.camera.segmentation_utils import read_ply_to_numpy, batch_segment_and_label
# Initialize YOLO model
model = YOLO("/home/capre/Point-Cloud-Stream/runs/segment/train6/weights/best.pt")

o3d_device = o3d.core.Device("CUDA:0")
path = Path("/home/capre/disk_4/yutao/leo_data/data_2nd")
resources_path = path / 'resources'
all_data = json.load(open(path / 'all_data.json'))
# Intrinsic matrix
intrinsic = np.array([
    [610.5961520662408, 0.0,               639.8919938587554],
    [0.0,               617.4130735412369, 358.3889735843055],
    [0.0,               0.0,               1.0]
], dtype=np.float32).T


def o3d_t_to_torch(o3d_t):
    return torch.utils.dlpack.from_dlpack(o3d_t.to_dlpack())


batch_pcds = []
batch_rgbs = []

batch_intrinsics = []
batch_extrinsics = []
batch_info = []  # (each_record, idx)
res_data = {}

record_keys = list(all_data.keys())
for each_record in tqdm(record_keys, desc='Calculate Seg Label'):
    current_pcds = []
    current_imgs = []
    current_intrs = []
    current_extrs = []
    current_info = []
    saved = True
    
    for idx, each_image in enumerate(all_data[each_record]['color_files']):
        try:
            color_path = resources_path / each_image
            point_cloud = read_ply_to_numpy(resources_path / all_data[each_record]['point_cloud_files'][idx])
            full_color = cv2.imread(color_path)
            current_pcds.append(point_cloud)
            current_imgs.append(full_color)
            current_extrs.append(np.eye(4))
            current_intrs.append(intrinsic)
            current_info.append([each_record, idx])
        except Exception as e:
            print(f"{each_image}", e)
            saved = False
            break
    if saved:
        batch_pcds.extend(current_pcds)
        batch_rgbs.extend(current_imgs)
        batch_extrinsics.extend(current_extrs)
        batch_intrinsics.extend(current_intrs)
        batch_info.extend(current_info)
        res_data[each_record] = all_data[each_record]




In [ ]:
from app.utils.camera.segmentation_utils import segment_image_with_yolo, label_point_cloud_from_segmentation


labels_list = batch_segment_and_label(model, current_pcds, 
                                      batch_rgbs, batch_intrinsics, batch_extrinsics)

for i, labels in tqdm(enumerate(labels_list), desc='Saving ply', total=len(labels_list)):
    print(np.count_nonzero(labels==80))
    record, idx = batch_info[i]
    pcd = batch_pcds[i]
    ply_name = resources_path / all_data[record]['point_cloud_files'][idx]
    # pcd = read_ply_to_numpy(ply_name)
    pcd[:, 6] = labels
    vertex = np.array(
        [(x_p, y_p, z_p, r_p, g_p, b_p, s_p) for x_p, y_p, z_p, r_p, g_p, b_p, s_p in pcd],
        dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
               ('red', 'f4'), ('green', 'f4'), ('blue', 'f4'), ('segment_id', 'i4')]
    )
    ply = PlyData([PlyElement.describe(vertex, 'vertex')], text=True)
    ply.write(str(ply_name))

In [ ]:
from ultralytics import YOLO
import numpy as np
model = YOLO("/home/capre/Point-Cloud-Stream/runs/segment/train6/weights/best.pt")
intrinsic = np.array([[624.8504238317045,   0.0,                657.0969388474656],
                        [0.0,               627.2424412728651,  346.1339921256911],
                        [0.0,               0.0,                1.0]]).T

In [ ]:
from app.utils.camera.segmentation_utils import segment_pcd_from_2d, read_ply_to_numpy
from plyfile import PlyData, PlyElement
import cv2

pcd = read_ply_to_numpy('/home/capre/disk_4/yutao/leo_data/data_2nd/resources/0acd5ab6fd3341ca8a26b6508f311a85_point_cloud_0.ply')
# image = cv2.imread('/home/capre/disk_4/yutao/leo_data/data_1st/resources/0a7ee4d0cf344e15a81c68be0be1fb96_color_3.png')
# labels = segment_pcd_from_2d(model, pcd, image, intrinsic)
np.max(pcd[:,3:6])

In [ ]:
import json
file_1 = '/home/capre/disk_4/yutao/leo_data/data_1st/all_data.json'
file_2 = '/home/capre/disk_4/yutao/leo_data/data_2nd/all_data.json'
json_1 = json.load(open(file_1))
json_2 = json.load(open(file_2))
print(len(json_1), len(json_2))
merged = {**json_1, **json_2}
json.dump(merged, open('/home/capre/disk_4/yutao/leo_data/all_data.json', 'w'), indent=4)

In [ ]:
from plyfile import PlyData, PlyElement
import numpy as np
def read_ply_to_numpy(file_path):
    pcd = PlyData.read(open(file_path, 'rb'))
    pcd_array = np.array(pcd.elements[0].data)
    points_array = np.vstack([
        pcd_array['x'], pcd_array['y'], pcd_array['z'], 
        pcd_array['red'], pcd_array['green'], pcd_array['blue'], 
        pcd_array['segment_id']
    ]).T
    return points_array


file = '/home/capre/disk_4/yutao/leo_data/merged_data/resources/0a7ee4d0cf344e15a81c68be0be1fb96_point_cloud_0.ply'

pcd = read_ply_to_numpy(file)
pcd[100]

# Statistic of validation

In [ ]:
import json
valid_json = "/home/capre/disk_4/yutao/leo_data/merged_data/validity.json"
# with open(valid_json, 'r') as f:
#     data = json.load(f)

def calculate_ratios(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
    
    valid_count = 0
    inverse_in_valid_count = 0
    total_count = 0
    inverse_count = 0
    
    for key, value in data.items():
        if 'valid' in value and 'inverse' in value:
            total_count += 1
            if value['inverse']:
                inverse_count += 1
            if value['valid']:
                valid_count += 1
                if value['inverse']:
                    inverse_in_valid_count += 1
    
    valid_ratio = valid_count / total_count if total_count > 0 else 0
    inverse_ratio_in_valid = inverse_in_valid_count / valid_count if valid_count > 0 else 0
    
    return {
        'total_count': total_count,
        'valid_count': valid_count,
        'inverse_count': inverse_count,
        'valid_ratio': valid_ratio,
        'inverse_ratio_in_valid': inverse_ratio_in_valid
    }

calculate_ratios(valid_json)

In [2]:
import json
path = "/home/capre/disk_4/yutao/leo_data/merged_data/res_data.json"

data:dict = json.load(open(path, 'r'))

In [3]:
prompt_set = set([data[x]['prompt'] for x in data])

In [10]:
prompt_map = {x:x for x in prompt_set}

In [42]:
# from pathlib import Path
# with open(Path(path).parent / 'prompt_map.json', 'w') as f:
#     json.dump(prompt_map, f, indent=2)
from pathlib import Path
with open(Path(path).parent / 'prompt_map.json', 'r') as f:
    prompt_map = json.load(f)

In [ ]:
import copy 
data_remap = copy.deepcopy(data)

for key in data_remap:
    data_remap[key]['prompt'] = prompt_map[data_remap[key]['prompt']]
data_remap

In [44]:
valid_js_path = Path(path).parent / 'validity.json'
with open(valid_js_path, 'r') as f:
    valid_json = json.load(f)
valid_json
res_json = {}
for key in valid_json:
    if valid_json[key]['valid'] == True:
        if valid_json[key]['inverse'] == True:
            if 'right' in data_remap[key]['prompt']:
                data_remap[key]['prompt'] = data_remap[key]['prompt'].replace('right', 'left')
            if 'left' in data_remap[key]['prompt']:
                data_remap[key]['prompt'] = data_remap[key]['prompt'].replace('left', 'right')
        res_json[key] = data_remap[key]



In [45]:
with open(valid_js_path.parent / 'refined_data.json', 'w') as f:
    json.dump(res_json, f, indent=4)

In [ ]:
len(res_json)